In [13]:


# path_inference_data = Path("/afs/crc.nd.edu/group/dmsquare/vol2/myu2/ComparisonSentences/experiments/pair_scoring/Q11424/data_2k_hn.csv")
# dataset_inference = pd.read_csv(path_inference_data)
# X_infer = dataset_inference.iloc[:, 1:-1]
# Y_infer = dataset_inference.iloc[:, -1]


18000 2000


In [24]:
from sklearn.metrics import r2_score, explained_variance_score, max_error, mean_absolute_error, mean_squared_error, mean_poisson_deviance
from sklearn.ensemble import RandomForestRegressor
# as classification / ranking
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, ndcg_score, f1_score
import numpy as np
import scipy.stats as stats
import sklearn
from sklearn.model_selection import train_test_split

from pathlib import Path
import pandas as pd

# train
path_data = Path("/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/global/data.tsv")
dataset = pd.read_csv(path_data)
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.1, random_state=0)
X_train = X_train.iloc[:, 1:]
X_dev = X_dev.iloc[:, 1:]
print(len(X_train), len(X_dev))

regressor = RandomForestRegressor(n_estimators=130, random_state=0)
regressor.fit(X_train, Y_train)
y_pred_train = regressor.predict(X_train)
print("train_r2", r2_score(Y_train, y_pred_train))
y_pred = regressor.predict(X_dev)



def print_evaluation(truths, preds):
    print("test_r2", r2_score(truths, preds))
    print("test_mae", mean_absolute_error(truths, preds))
    print("test_mse", mean_squared_error(truths, preds))
    print("ndcg", ndcg_score(np.array([truths]), [preds]))
    print("kendalltau, p", stats.kendalltau(truths, preds))
    y_pred = [1 if x > math.log(1.5) else 0 for x in preds]
    y_label = [1 if x > math.log(1.5) else 0 for x in truths]
    print("acc",accuracy_score(y_label, y_pred))
    print("p",precision_score(y_label, y_pred))
    print("r", recall_score(y_label, y_pred))
    print("f1", f1_score(y_label, y_pred))

print_evaluation(Y_dev, y_pred)
print(f"Importance of {len(regressor.feature_importances_)} features:")
for k,v in zip(regressor.feature_names_in_, regressor.feature_importances_):
    print(k,v)





FileNotFoundError: [Errno 2] No such file or directory: '/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/global/data.tsv'

In [23]:
# test
path_test_data = Path("/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/Q5/test_data.tsv")
test_dataset = pd.read_csv(path_test_data)
X_test = test_dataset.iloc[:, 1:-1]
Y_test = test_dataset.iloc[:, -1]

y_pred_test = regressor.predict(X_test)
print_evaluation(Y_test, y_pred_test)


test_r2 0.48569902937838993
test_mae 0.21071987566117553
test_mse 0.12806589675439434
ndcg 0.9280036815107774
kendalltau, p KendalltauResult(correlation=0.6399173565189493, pvalue=0.0)
acc 0.9070903549659376
p 0.921954845303354
r 0.8894765148798852
f1 0.9054245175418586


In [18]:
for t in y_pred_test:
    print(t)

0.9519173804071424
0.7650062147215685
0.8098557039199262
0.1768587691856505
0.7792081197562772
0.9549675648848185
0.7892088980020999
0.3461726979635187
0.8096559073554888
0.3633884408239531
1.1521691664896436
1.210884667568511
0.952723541502009
1.0166510217505849
1.1236959376926516
1.3183085621892598
0.9489214405797161
0.8467781213259309
1.316528388694007
1.0205202072230268
0.8243216558253378
0.8523572680331347
0.5051025750725485
0.4350029748727981
0.38621407773958616
0.7771811948803212
1.0753124598133794
1.0186477922870827
0.7648833150713905
1.1508694103832602
0.6373185740537003
1.0107307464480753
0.9594647165949554
0.8273854621228245
1.1061629319117046
0.3862501004157737
0.873360731356285
0.9943967178395192
0.9385923202860245
1.2058972258586464
0.5110607487036672
0.9223263276706184
0.29416059974194886
0.5264594277053511
0.8603129423176809
0.9215470259761852
0.7567464666548683
0.21056666637544952
1.1258803581969639
0.8354435205215595
1.5144705866530868
1.0029771642128549
1.12267749589

Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV
param_search = {'n_estimators': range(10, 201, 20)}
gsearch = GridSearchCV(estimator=RandomForestRegressor(random_state=0),
                       param_grid=param_search,
                       scoring="r2", 
                       cv=3 )
gsearch.fit(X_train, Y_train)
gsearch.cv_results_, gsearch.best_params_, gsearch.best_score_

({'mean_fit_time': array([ 1.43733134,  4.36580062,  7.43976398, 10.23190761, 13.21465816,
         15.57780185, 18.71514196, 21.98928514, 25.5790637 , 28.58662133]),
  'std_fit_time': array([0.03607972, 0.1024238 , 0.34429086, 0.32677396, 0.47755168,
         0.2011114 , 0.22102966, 0.24217953, 0.92565247, 0.40305405]),
  'mean_score_time': array([0.00780578, 0.01873102, 0.03128424, 0.04254565, 0.0522965 ,
         0.06116676, 0.07560439, 0.08869953, 0.09998827, 0.11612582]),
  'std_score_time': array([0.00114294, 0.00131833, 0.0014688 , 0.00399869, 0.00203292,
         0.00092063, 0.00515382, 0.00797094, 0.00694944, 0.00739642]),
  'param_n_estimators': masked_array(data=[10, 30, 50, 70, 90, 110, 130, 150, 170, 190],
               mask=[False, False, False, False, False, False, False, False,
                     False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 10},
   {'n_estimators': 30},
   {'n_estimators': 50},
   {'n_estimators':

In [ ]:
param_test2 = {'max_depth':range(3,20,4), 'min_samples_split':range(10,201,30)}

gsearch = GridSearchCV(estimator=RandomForestRegressor(n_estimators=130, random_state=10),
                       param_grid=param_test2,
                       scoring="r2", 
                       cv=3 )
gsearch.fit(X_train, Y_train)
gsearch.cv_results_, gsearch.best_params_, gsearch.best_score_


In [ ]:
# test in another domain
regressor = RandomForestRegressor(n_estimators=130, random_state=0)
regressor.fit(X_train, Y_train)
y_pred_train = regressor.predict(X_train)
print("train_r2", r2_score(Y_train, y_pred_train))
Y_pred = regressor.predict(X_test)

# as classification
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, ndcg_score
print("ndcg", ndcg_score(np.array([Y_test]), [Y_pred]))
y_pred = [1 if x > math.log(1.5) else 0 for x in Y_pred]
y_label = [1 if x > math.log(1.5) else 0 for x in Y_test]
print("acc",accuracy_score(y_label, y_pred))
print("p",precision_score(y_label, y_pred))
print("r", recall_score(y_label, y_pred))

In [ ]:
precisions = []
recalls = []
for i in range(10):
    k = 0.1*i
    pred = [1 if x > math.log(1 + k) else 0 for x in y_pred]
    label = [1 if x > math.log(1.5) else 0 for x in Y_test]
    precisions.append(precision_score(label, pred))
    recalls.append(recall_score(label, pred))
print(precisions)
print(recalls)

In [ ]:
Y_pred

In [ ]:
X = dataset.iloc[:, 1:-1]
Y = dataset.iloc[:, -1]

train_score = []
test_score = []
n = 10
num_train_example = int(len(X_train) / n) 
for i in range(n):
    start = i*num_train_example
    end = (i+1)*num_train_example
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train[:end], Y_train[:end])
    y_train_pred = regressor.predict(X_train[:end])
    train_score.append(r2_score(Y_train[:end], y_train_pred))

    y_test_pred = regressor.predict(X_test)
    test_score.append(r2_score(Y_test, y_test_pred))
    
import matplotlib.pyplot as plt
plt.plot([i for i in range(n)], train_score, label="train")
plt.plot([i for i in range(n)], test_score, label="test")
plt.legend()
plt.show()